#### LightGBMでベースラインモデル構築
#### LB: 386.23556
 - ラベルエンコードのみ適用
 - ホールドアウト法

In [1]:
import os
os.chdir('/Users/kinoshitashouhei/Desktop/competitions/05_Prob_Space/Kiva/')

In [7]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import gc
import lightgbm as lgb
import optuna.integration.lightgbm as lgb_optuna
import pickle
import optuna
import warnings

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, train_test_split
from sklearn.pipeline import Pipeline

# 自作関数のインポート
from functions.common import *
from functions import preprocessing

warnings.filterwarnings('ignore')

In [3]:
# 各種設定
N_COMP = 10
DICT_DTYPE = {'LOAN_ID': 'str', 'IMAGE_ID': 'str'}
LIST_COL_DROPS = [COL_LOAN_ID,
                  COL_DESCRIPTION,
                  COL_DESCRIPTION_TRANSLATED,
                  COL_ACTIVITY_NAME,
                  COL_LOAN_USE,
                  COL_IMAGE_ID,
                  COL_TOWN_NAME,
                  COL_COUNTRY_NAME,
                  COL_TAGS]

LIST_TARGET_ENC = [COL_ORIGINAL_LANGUAGE]
LIST_LABEL_ENC = [COL_SECTOR_NAME, COL_CURRENCY_POLICY, COL_CURRENCY, COL_REPAYMENT_INTERVAL, COL_DISTRIBUTION_MODEL]
LIST_LDA_DESCRIPTINO_TRANSLATED = [f'COL_LDA_DESCRIPTION_TRANSLATED_{i+1}' for i in range(N_COMP)]
LIST_LDA_LOAN_USE = [f'COL_LDA_LOAN_USE_{i+1}' for i in range(N_COMP)]

In [4]:
# モデルのパス
LDA_PATH_DESCRIPTION_TRANSLATED = 'models/lda_description_translated.pkl'
LDA_PATH_LOAN_USE = 'models/lda_loan_use.pkl'

# モデルの読み込み
with open(LDA_PATH_DESCRIPTION_TRANSLATED, mode='rb') as fp:
    CLF_LDA_DESCRIPTION_TRANSLATED = pickle.load(fp)
    
with open(LDA_PATH_LOAN_USE, mode='rb') as fp:
    CLF_LDA_LOAN_USE = pickle.load(fp)

In [5]:
# csvデータの読み込み
df_train = pd.read_csv('data/train.csv', dtype=DICT_DTYPE)
df_test = pd.read_csv('data/test.csv', dtype=DICT_DTYPE)

In [8]:
# ラベルエンコードを適用
LIST_LABEL_ENC = [COL_ORIGINAL_LANGUAGE, COL_SECTOR_NAME, COL_CURRENCY_POLICY, COL_CURRENCY, COL_REPAYMENT_INTERVAL, COL_DISTRIBUTION_MODEL, COL_COUNTRY_CODE]
df_train, df_test = preprocessing.label_encoding(df_train, df_test, LIST_LABEL_ENC)

In [9]:
df_train.head()

,LOAN_ID,ORIGINAL_LANGUAGE,DESCRIPTION,DESCRIPTION_TRANSLATED,LOAN_AMOUNT,IMAGE_ID,ACTIVITY_NAME,SECTOR_NAME,LOAN_USE,COUNTRY_CODE,COUNTRY_NAME,TOWN_NAME,CURRENCY_POLICY,CURRENCY_EXCHANGE_COVERAGE_RATE,CURRENCY,TAGS,REPAYMENT_INTERVAL,DISTRIBUTION_MODEL
0,1733169,0,Teodora is a 50-year-old married woman from th...,Teodora is a 50-year-old married woman from th...,100,3115271,Weaving,1,"to purchase materials like nipa palm, bamboo ...",39,Philippines,"Maribojoc, Bohol",0,0.1,34,#Elderly,2,1
1,1546998,0,Diego is 32 years old and lives in the municip...,Diego is 32 years old and lives in the municip...,1350,2870403,Barber Shop,12,"to buy two hair clippers, a new barber chair, ...",7,Colombia,Apartadó,0,0.1,4,"user_favorite, user_favorite",2,1
2,1808517,4,"Osman, es un joven de 27 años de edad, soltero...","Osman is a young man, 27 years old, single, an...",225,3215705,Farming,0,to purchase sacks of fertilizers to care for a...,16,Honduras,"Nueva Frontera, Santa Barbara.",0,0.1,13,NaN,0,1
3,1452940,0,"His name is Nino, 31 years old, married to Che...","His name is Nino, 31 years old, married to Che...",350,2745031,Motorcycle Transport,13,"to pay for fuel, tires and change oil for his ...",39,Philippines,"Silang, Cavite",0,0.1,34,user_favorite,2,1
4,1778420,0,"Pictured above is Teresa, often described as a...","Pictured above is Teresa, often described as a...",625,3083800,Farming,0,to purchase hybrid seeds and fertilizer to imp...,22,Kenya,Mumias,0,0.1,19,"#Eco-friendly, #Sustainable Ag, #Parent, #Elde...",0,1


In [11]:
# 不要な列をドロップ
df_train.drop(columns=LIST_COL_DROPS, inplace=True)
df_test.drop(columns=LIST_COL_DROPS, inplace=True)

In [12]:
df_train.head()

,ORIGINAL_LANGUAGE,LOAN_AMOUNT,SECTOR_NAME,COUNTRY_CODE,CURRENCY_POLICY,CURRENCY_EXCHANGE_COVERAGE_RATE,CURRENCY,REPAYMENT_INTERVAL,DISTRIBUTION_MODEL
0,0,100,1,39,0,0.1,34,2,1
1,0,1350,12,7,0,0.1,4,2,1
2,4,225,0,16,0,0.1,13,0,1
3,0,350,13,39,0,0.1,34,2,1
4,0,625,0,22,0,0.1,19,0,1


In [13]:
# 学習に使用するデータを設定
X = df_train.drop(columns=[COL_LOAN_AMOUNT])
y = df_train[COL_LOAN_AMOUNT]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train) 

In [14]:
# ベストなパラメータ、途中経過を保存する
params = {
    'objective': 'regression',
    'metric': 'mae',
    "verbosity": -1,
    "boosting_type": "gbdt",
}

best_params, history = {}, []

# LightGBM学習
gbm = lgb_optuna.train(params,
                lgb_train,
                num_boost_round=200,
                valid_sets=[lgb_train, lgb_eval],
                early_stopping_rounds=50
               )

best_params = gbm.params

[I 2021-12-27 12:43:39,410] A new study created in memory with name: no-name-bd192fa4-41a0-4883-bdab-3aeb5e53fc0b
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[1]	valid_0's l1: 532.135	valid_1's l1: 528.128
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l1: 506.747	valid_1's l1: 502.811
[3]	valid_0's l1: 483.991	valid_1's l1: 480.2
[4]	valid_0's l1: 465.548	valid_1's l1: 462.312
[5]	valid_0's l1: 448.923	valid_1's l1: 446.144
[6]	valid_0's l1: 434.923	valid_1's l1: 432.462
[7]	valid_0's l1: 422.522	valid_1's l1: 420.894
[8]	valid_0's l1: 412.417	valid_1's l1: 411.417
[9]	valid_0's l1: 404.046	valid_1's l1: 403.222
[10]	valid_0's l1: 396.329	valid_1's l1: 396.052
[11]	valid_0's l1: 389.069	valid_1's l1: 389.074
[12]	valid_0's l1: 383.799	valid_1's l1: 383.811
[13]	valid_0's l1: 379.016	valid_1's l1: 379.329
[14]	valid_0's l1: 374.394	valid_1's l1: 375.016
[15]	valid_0's l1: 370.968	valid_1's l1: 371.807
[16]	valid_0's l1: 367.305	valid_1's l1: 368.217
[17]	valid_0's l1: 363.441	valid_1's l1: 364.464
[18]	valid_0's l1: 360.758	valid_1's l1: 361.993
[19]	valid_0's l1: 358.055	valid_1's l1: 359.22
[20]	valid_0's l1: 3

feature_fraction, val_score: 324.067047:  14%|#4        | 1/7 [00:00<00:04,  1.28it/s]

[195]	valid_0's l1: 318.054	valid_1's l1: 324.129
[196]	valid_0's l1: 318.045	valid_1's l1: 324.142
[197]	valid_0's l1: 318.034	valid_1's l1: 324.113
[198]	valid_0's l1: 318	valid_1's l1: 324.101
[199]	valid_0's l1: 317.977	valid_1's l1: 324.09
[200]	valid_0's l1: 317.948	valid_1's l1: 324.067
Did not meet early stopping. Best iteration is:
[200]	valid_0's l1: 317.948	valid_1's l1: 324.067
[1]	valid_0's l1: 532.135	valid_1's l1: 528.128
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l1: 506.747	valid_1's l1: 502.811
[3]	valid_0's l1: 483.991	valid_1's l1: 480.2
[4]	valid_0's l1: 465.548	valid_1's l1: 462.312
[5]	valid_0's l1: 448.923	valid_1's l1: 446.144
[6]	valid_0's l1: 434.923	valid_1's l1: 432.462
[7]	valid_0's l1: 422.522	valid_1's l1: 420.894
[8]	valid_0's l1: 412.417	valid_1's l1: 411.417
[9]	valid_0's l1: 404.046	valid_1's l1: 403.222
[10]	valid_0's l1: 396.329	valid_1's l1: 396.052
[11]	valid_0's l1: 389.069	valid_1's l1: 389.074
[12]	valid_0's l1:

feature_fraction, val_score: 324.067047:  29%|##8       | 2/7 [00:01<00:03,  1.57it/s]

...

min_data_in_leaf, val_score: 321.742806:  80%|########  | 4/5 [00:05<00:01,  1.41s/it]

[196]	valid_0's l1: 315.774	valid_1's l1: 322.565
[197]	valid_0's l1: 315.766	valid_1's l1: 322.571
[198]	valid_0's l1: 315.748	valid_1's l1: 322.56
[199]	valid_0's l1: 315.786	valid_1's l1: 322.604
[200]	valid_0's l1: 315.768	valid_1's l1: 322.593
Did not meet early stopping. Best iteration is:
[198]	valid_0's l1: 315.748	valid_1's l1: 322.56
[1]	valid_0's l1: 528.374	valid_1's l1: 524.749
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l1: 498.275	valid_1's l1: 495.174
[3]	valid_0's l1: 472.144	valid_1's l1: 469.68
[4]	valid_0's l1: 450.054	valid_1's l1: 448.374
[5]	valid_0's l1: 430.837	valid_1's l1: 429.843
[6]	valid_0's l1: 415.961	valid_1's l1: 415.332
[7]	valid_0's l1: 401.884	valid_1's l1: 402.041
[8]	valid_0's l1: 389.893	valid_1's l1: 390.663
[9]	valid_0's l1: 380.678	valid_1's l1: 381.775
[10]	valid_0's l1: 371.856	valid_1's l1: 373.366
[11]	valid_0's l1: 364.381	valid_1's l1: 366.299
[12]	valid_0's l1: 358.196	valid_1's l1: 360.464
[13]	valid_0's 

min_data_in_leaf, val_score: 321.742806: 100%|##########| 5/5 [00:07<00:00,  1.46s/it]

[192]	valid_0's l1: 314.1	valid_1's l1: 322.658
[193]	valid_0's l1: 314.058	valid_1's l1: 322.542
[194]	valid_0's l1: 314.037	valid_1's l1: 322.454
[195]	valid_0's l1: 314.019	valid_1's l1: 322.38
[196]	valid_0's l1: 313.991	valid_1's l1: 322.29
[197]	valid_0's l1: 314.01	valid_1's l1: 322.291
[198]	valid_0's l1: 314.015	valid_1's l1: 322.304
[199]	valid_0's l1: 314.02	valid_1's l1: 322.351
[200]	valid_0's l1: 314.019	valid_1's l1: 322.361
Did not meet early stopping. Best iteration is:
[196]	valid_0's l1: 313.991	valid_1's l1: 322.29


{'objective': 'regression',
 'metric': 'l1',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'feature_pre_filter': False,
 'lambda_l1': 5.85190303727203e-06,
 'lambda_l2': 5.794048474265727,
 'num_leaves': 132,
 'feature_fraction': 0.8999999999999999,
 'bagging_fraction': 0.7346168634945545,
 'bagging_freq': 4,
 'min_child_samples': 5,
 'num_iterations': 200,
 'early_stopping_round': 50}

In [15]:
best_params

{'objective': 'regression',
 'metric': 'l1',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'feature_pre_filter': False,
 'lambda_l1': 5.85190303727203e-06,
 'lambda_l2': 5.794048474265727,
 'num_leaves': 132,
 'feature_fraction': 0.8999999999999999,
 'bagging_fraction': 0.7346168634945545,
 'bagging_freq': 4,
 'min_child_samples': 5,
 'num_iterations': 200,
 'early_stopping_round': 50}

In [20]:
params = {'objective': 'regression',
          'metric': 'l1',
          'verbosity': -1,
          'boosting_type': 'gbdt',
          'feature_pre_filter': False,
          'lambda_l1': 5.85190303727203e-06,
          'lambda_l2': 5.794048474265727,
          'num_leaves': 132,
          'feature_fraction': 0.8999999999999999,
          'bagging_fraction': 0.7346168634945545,
          'bagging_freq': 4,
          'min_child_samples': 5,
          'num_iterations': 200,
          'early_stopping_round': 50}

In [17]:
# モデルの学習
model = lgb.train(params,
                  train_set=lgb_train, # トレーニングデータの指定
                  valid_sets=lgb_eval, # 検証データの指定
                  )

[1]	valid_0's l1: 524.484
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l1: 494.63
[3]	valid_0's l1: 469.115
[4]	valid_0's l1: 447.752
[5]	valid_0's l1: 429.303
[6]	valid_0's l1: 414.779
[7]	valid_0's l1: 401.276
[8]	valid_0's l1: 389.846
[9]	valid_0's l1: 380.911
[10]	valid_0's l1: 372.399
[11]	valid_0's l1: 365.175
[12]	valid_0's l1: 359.286
[13]	valid_0's l1: 353.947
[14]	valid_0's l1: 349.335
[15]	valid_0's l1: 345.668
[16]	valid_0's l1: 342.466
[17]	valid_0's l1: 339.868
[18]	valid_0's l1: 337.632
[19]	valid_0's l1: 335.69
[20]	valid_0's l1: 333.991
[21]	valid_0's l1: 332.489
[22]	valid_0's l1: 331.171
[23]	valid_0's l1: 330.077
[24]	valid_0's l1: 329.138
[25]	valid_0's l1: 328.279
[26]	valid_0's l1: 327.545
[27]	valid_0's l1: 326.969
[28]	valid_0's l1: 326.443
[29]	valid_0's l1: 325.891
[30]	valid_0's l1: 325.435
[31]	valid_0's l1: 325.087
[32]	valid_0's l1: 324.757
[33]	valid_0's l1: 324.579
[34]	valid_0's l1: 324.442
[35]	valid_0's l1: 324.372
[36]	

In [19]:
sample_sub = pd.read_csv('data/sample_submission.csv')
preds = model.predict(df_test)
sample_sub[COL_LOAN_AMOUNT] = preds

sample_sub.to_csv('submit/sample_sub_base_line.csv', index=None)